# Vertrauliche Vorhersagen mit Syft Keras

## Schritt 3: Vertrauliche Vorhersagen mit Syft Keras - Serving (Client)

Herzlichen Glückwunsch! Nachdem Sie ihr Model mit normalem Keras traininert und mit Syft Keras abgesichert haben, sind Sie nun bereit einige vertrauliche Vorhersagen zu bearbeiten.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist

import syft as sy

## Daten

Hier bereiten wir den MNIST Datensatz auf. Dies ist identisch zu der Aufbereitung für das Training.

In [ ]:
# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

## Verbinden mit dem Model

Bevor das Model befragt werden kann, muss eine Verbindung zu ihm hergestellt werden. Dafür wird ein Client eingesetzt. Anschließend müssen genau dieselben drei TFEWorker (`alice`, `bob`, und `carol`) und das Cluster definiert werden. Zu guter Letzt rufen Sie `connect_to_model` auf. Dies erstellt einen TFE Queue-Server auf der Client-Seite, welcher sich mit dem Queue-Server von `model.serve()` aus **Teil 13b** verbindet. Der Client-Queue-Server ist verantwortlich dafür die Rohdaten geheim aufzuteilen bevor er eine Vorhersage-Anfrage stellt.

In [ ]:
num_classes = 10
input_shape = (1, 28, 28, 1)
output_shape = (1, num_classes)

In [ ]:
client = sy.TFEWorker()

alice = sy.TFEWorker(host='localhost:4000')
bob = sy.TFEWorker(host='localhost:4001')
carol = sy.TFEWorker(host='localhost:4002')
cluster = sy.TFECluster(alice, bob, carol)

client.connect_to_model(input_shape, output_shape, cluster)

## Abfragen des Models

Nun sind Sie bereit um vertrauliche Vorhersagen zu erhalten! Das Aufrufen von `query_model` fügt das `image` in die oben erstellte Queue hinzu. Diese teilt die Daten lokal geheim auf und übermittelt diese Teile anschließend dem Model-Server aus **Teil 13b**.

In [ ]:
# User inputs
num_tests = 3
images, expected_labels = x_test[:num_tests], y_test[:num_tests]

In [ ]:
for image, expected_label in zip(images, expected_labels):

    res = client.query_model(image.reshape(1, 28, 28, 1))
    predicted_label = np.argmax(res)

    print("Das Bild hat das Label {} und wurde {} klassifiziert als {}".format(
        expected_label,
        "richtig" if expected_label == predicted_label else "falsch",
        predicted_label))

Das lief klasse. Alle drei Bilder konnten richtig klassifiziert werden! Das Besondere an diesen Klassifizierungen ist, dass Sie keinerlei geheime Informationen offen legen mussten, um den Service erhalten zu können. Der Model-Host sah zu keiner Zeit Ihre Eingabe-Daten noch Ihre Ausgabe-Klassifizierungen und auch mussten Sie das Model nicht herunterladen.  
Sie konnten geheime Vorhersagen auf verschlüsselten Daten mit einem verschlüsselten Model erhalten!

Befor wir aufbrechen und dies in unseren eigenen Apps anwenden, lassen Sie uns noch schnell zurück zu **Teil 13b** gehen und dort unser bereit gestelltes Model aufräumen!